# Data wrangling with SQL

## OpenStreetMap Project 

In [7]:
#!/usr/bin/env python 
# -*- coding: utf-8 -*- 

import xml.etree.cElementTree as ET
from collections import defaultdict
import pprint
import re
import codecs
import cerberus
import schema
import csv
import sqlite3

In [8]:
filename="LondonUK.osm"

### Count Tags

In [9]:
# Parse osm to count unique name tags
def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename):
        if elem.tag not in tags:
            tags[elem.tag] = 1
        else:
            tags[elem.tag] += 1
    return tags

Uni_tags = count_tags(filename)
pprint.pprint(Uni_tags)

{'bounds': 1,
 'member': 96894,
 'nd': 1499822,
 'node': 1113451,
 'osm': 1,
 'relation': 6235,
 'tag': 1263929,
 'way': 208318}


### Inspect Tag types

In [10]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# Count 4 unique tag categories 
def key_type(element, keys):
    if element.tag == "tag":
        if lower.search(element.attrib['k']): 
            keys["lower"] += 1
        elif lower_colon.search(element.attrib['k']):
            keys['lower_colon'] += 1        
        elif problemchars.search(element.attrib['k']):               
                keys['problemchars'] += 1
        else:        
                keys['other'] += 1 
    pass
    return keys

# Parse osm to get the count values of unique tag categories 
def process_map_tags(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys


Type_tags = process_map_tags(filename)
pprint.pprint(Type_tags)

{'lower': 1011731, 'lower_colon': 226272, 'other': 25925, 'problemchars': 1}


### Inspect Unique Users

In [11]:
def get_user(element):
    return


def process_map_users(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if 'uid' in element.attrib:
            users.add(element.attrib['uid'])
        pass
    return users


Uni_users = process_map_users(filename)
pprint.pprint(Uni_users)

set(['1',
     '100214',
     '1002580',
     '100597',
     '100643',
     '101150',
     '1012721',
     '1013024',
     '10136',
     '101374',
     '1015446',
     '1016290',
     '102002',
     '1026712',
     '102736',
     '1028396',
     '10291',
     '1030',
     '1031302',
     '103253',
     '103280',
     '10345',
     '1036454',
     '104101',
     '104255',
     '104375',
     '104459',
     '104519',
     '104640',
     '104962',
     '1051637',
     '105267',
     '105321',
     '1058129',
     '105831',
     '105998',
     '1061178',
     '106181',
     '1065603',
     '1066089',
     '1067273',
     '106786',
     '1072131',
     '1072240',
     '107237',
     '107257',
     '1074589',
     '107507',
     '107565',
     '10785',
     '108075',
     '10857',
     '1089198',
     '1089595',
     '108964',
     '109205',
     '10927',
     '109437',
     '109472',
     '10983',
     '109925',
     '1100566',
     '110203',
     '110263',
     '110632',
     '110792',
   

In [12]:
len(Uni_users)

3035

### Audit Street Names

In [13]:
osmfile = "LondonUK.osm"
 
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
street_types = defaultdict(set)

'''
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]
'''
# After auditing the street tags, I return back and expand my list of expected with more valid street types and following this I created a mapping dict based on the rest not expected types of streets.
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons","Close","Crescent","Gardens","Grove","Hill","Mews","Row",
            "Terrace","Walk","Way","Park", "Gate", "Market","Parade"]

# If street type not in expected add it to dictionary
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
             
# Shows whether a tag contains a street name 
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street") 


# Parse an element if this equals "way", return all it's subtags named "tag"
def audit_street(filename):
    for event, elem in ET.iterparse(filename, events=("start",)):
        if elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    return (dict(street_types))

Not_exp_street = audit_street(filename)
pprint.pprint(Not_exp_street)

{'263': set(['263']),
 '7NN': set(['N7 7NN']),
 '9EJ': set(['SE10 9EJ']),
 'Acre': set(['Long Acre']),
 'Aldgate': set(['Aldgate']),
 'Aldwych': set(['Aldwych']),
 'Alley': set(['Frying Pan Alley', 'Graces Alley']),
 'Approach': set(['Basin Approach',
                  'Blackwall Tunnel Northern Approach',
                  'College Approach',
                  'Global Approach']),
 'Arch': set(['Marble Arch']),
 'Axe': set(['Saint Mary Axe']),
 'Bailey': set(['Old Bailey']),
 'Bankside': set(['Bankside']),
 'Bedfordbury': set(['Bedfordbury']),
 'Bishopsgate': set(['Bishopsgate']),
 'Boltons': set(['The Boltons']),
 'Bondway': set(['Bondway']),
 'Bridge': set(['Blackfriars Bridge',
                'Kilburn Bridge',
                'London Bridge',
                'Lord Hills Bridge',
                'Wallis Bridge']),
 'Broadway': set(['Broadway', 'Deptford Broadway']),
 'Brunswick': set(['The Brunswick']),
 'Buildings': set(['Breams Buildings',
                   'Medway Buildings',
 

### Update Street Names

In [14]:
# Dictionary to correct audited mistakes
mapping = { "St": "Street",
            "street": "Street",
            "Rd": "Road"
            }

# Returns a fixed name where appropriate
def update_name(name, mapping):
    m = street_type_re.search(name)    
    if m:
        street_type = m.group()
        if street_type in mapping: 
            name = re.sub(street_type, mapping[street_type], name)
    return name

# Checking which names where corrected
for st_type, ways in audit_street(osmfile).iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        if  name <> better_name:   
            print  name,"=>", better_name

Peckham Rd => Peckham Road
Bonfield Rd => Bonfield Road
East Ferry Rd => East Ferry Road
Drummond street => Drummond Street
Rose street => Rose Street
Loman street => Loman Street
Brandon street => Brandon Street
Coptic street => Coptic Street
Pickfords Wharf, Clink St => Pickfords Wharf, Clink Street
Leonard St => Leonard Street
Collingwood St => Collingwood Street
Massingham St => Massingham Street
Elsa St => Elsa Street


## Audit Phone numbers

In [16]:

def is_phone_number(elem):
    return (elem.tag == 'tag') and (elem.attrib['k'] == 'phone')

def audit_phone(phone_in):
    # format all phone numbers to +4400000000
    
    # Remove symbols: "-", "(", ")"
    new_phone_number = "".join(phone_in.replace(' ','').replace('-','').replace('(','').replace(')','').split())

    # Fix the numbers starting with +    
    if new_phone_number[0]=="+":        
        if(new_phone_number)[0:5] == "+4420":
            # Remove any further than digits than the expected 
            up_phone_number = new_phone_number[0:13]
            print  new_phone_number,"=>", up_phone_number

        elif(new_phone_number)[0:5] == "+4402":
            up_phone_number="+" + "44" + new_phone_number[4:14]
            print  new_phone_number,"=>", up_phone_number

    #Fix the numbers not beggining with a "+"
    else:
        if(new_phone_number)[0:4] == "4420":
            up_phone_number="+" + new_phone_number[0:10]
            print  new_phone_number,"=>", up_phone_number

        if(new_phone_number)[0:3] == "020":
            up_phone_number="+" + "44" + new_phone_number[1:11]
            print  new_phone_number,"=>", up_phone_number
            
        #Fix the numbers beggining with "00" insted of "+"
        elif(new_phone_number)[0:2] == "00":
            up_phone_number="+" + new_phone_number[2:14]
            print  new_phone_number,"=>", up_phone_number

        else:
            if len (new_phone_number)==14:
                up_phone_number=new_phone_number[1:14]
                print  new_phone_number,"=>", up_phone_number

           
            
def audit(osmfile_in):
    osmfile = open(osmfile_in, 'r')
    for event, elem in ET.iterparse(osmfile_in):
        if is_phone_number(elem):
                #print elem.attrib['v']
                audit_phone(elem.attrib['v'])

if __name__ == '__main__':
    audit("LondonUK.osm")

 02073873033 => +442073873033
+442077230143 => +442077230143
+442078312660 => +442078312660
+442078362347 => +442078362347
02089854562 => +442089854562
02073875277 => +442073875277
02072757640 => +442072757640
+442079352361 => +442079352361
+442077238391 => +442077238391
00442076139800 => +442076139800
+442076361127 => +442076361127
02072474926 => +442072474926
+442079767407 => +442079767407
+442088529953 => +442088529953
+442086924033 => +442086924033
+4402083318000 => +442083318000
02072880572 => +442072880572
02074840736 => +442074840736
+442079760339 => +442079760339
02072675941 => +442072675941
+442074989633 => +442074989633
02074883686 => +442074883686
+442079282435 => +442079282435
+442072361285 => +442072361285
02072532944 => +442072532944
+442072425669 => +442072425669
+442078360291 => +442078360291
+442077900077 => +442077900077
‎+442070426900 => +442070426900
02078209310 => +442078209310
+442072240055 => +442072240055
02075316166 => +442075316166
02079871470 => +442079871470

### Preparing for Database - SQL

In [ ]:
OSM_PATH = "LondonUK.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']



def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    

    # Top Tag is 'node'
    if element.tag == 'node':

        # 'node' field
        for node in NODE_FIELDS:
            try:
                node_attribs[node] = element.attrib[node]
            except:
                # see what the issue is, remove when done
                print(node)
                # if the attribute doesn't exist, fill it with something
                node_attribs[node] = '000'
            
            
        # 'node_tags' field - Secondary tags
        for sec_tag in element:
            node_tag = {}
            node_tag['id'] = element.attrib['id']
            node_tag['value'] = sec_tag.attrib['v']

            # Tag 'k' value contains problematic characters
            if PROBLEMCHARS.match(sec_tag.attrib['k']):
                continue

            # Colon is present 
            elif ':' in sec_tag.attrib['k']:

                # The characters after the = tag key
                node_tag['key'] = sec_tag.attrib['k'].split(':',2)[1]
                # The characters before the = tag type
                node_tag['type'] = sec_tag.attrib['k'].split(':',2)[0]
                tags.append(node_tag)

            # No Colon present
            else:
                node_tag['key'] = sec_tag.attrib['k']
                node_tag['type'] = 'regular'
                tags.append(node_tag)

        return {'node': node_attribs, 'node_tags': tags}

    # Top Tag is 'way'
    elif element.tag == 'way': 

        # "node" field
        for node in WAY_FIELDS:
            way_attribs[node] = element.attrib[node]
        pos = 0

        # 'node_tags' field - Secondary tags
        for sec_tag in element:                                        
            if sec_tag.tag == 'tag':                                                     
                way_tag = {}
                way_tag['id'] = element.attrib['id']
                way_tag['value'] = sec_tag.attrib['v']
                # Tag "k" value contains problematic characters
                if PROBLEMCHARS.match(sec_tag.attrib['k']):
                    continue
                # Colon is present 
                elif ':' in sec_tag.attrib['k']:
                    # The characters after the = tag key
                    way_tag['key'] = sec_tag.attrib['k'].split(':',1)[1]
                    # The characters before the = tag type
                    way_tag['type'] = sec_tag.attrib['k'].split(':',2)[0]
                    tags.append(way_tag)
                # No Colon present
                else:
                    way_tag['key'] = sec_tag.attrib['k']
                    way_tag['type'] = 'regular'
                    tags.append(way_tag)

            elif sec_tag.tag == 'nd':                       
                way_node = {}
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = sec_tag.attrib['ref']
                way_node['position'] = pos
                pos += 1
                way_nodes.append(way_node)  

        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}



# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)